In [1]:
##### importing the libraries
import os
from distutils.util import strtobool
import pandas as pd
import pyodbc
from sqlalchemy import create_engine, update
from urllib.parse import quote_plus
from sqlalchemy import MetaData,Table
from sqlalchemy.sql import text as txt

In [2]:
conn =  "DRIVER={SQL Server};SERVER=AAPL-L-627\SQLEXPRESS;DATABASE=test"
quoted = quote_plus(conn)
new_con = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
conn = create_engine(new_con)

In [13]:
data = pd.read_excel(r'D:\1551091150_Sales Data.xlsx')

In [9]:
data.head()
data.shape

(29999, 23)

In [14]:
data.to_sql(name = 'ModelRunMeta',con = conn,if_exists='replace',index = False)

In [9]:
def connect_to_sql_server(server_name, database_name):

    cnxn_str = ("Driver={SQL Server Native Client 11.0};"
                "Server=%s;"
                "Database=%s;"
                "Trusted_Connection=yes;" % (server_name, database_name))

    cnxn = pyodbc.connect(cnxn_str)
    cursor = cnxn.cursor()

    return cursor

In [10]:
cursor = connect_to_sql_server('AAPL-L-627\SQLEXPRESS','test')

In [11]:
### function to delete the records through stored procedure

def delete(sql,cur,param):
    cur.execute(query,param)
    cur.commit()

In [12]:
#### query to execute the store procedure with parameters
query = """EXEC del @DBName = ?, @SchemaName = ?,@TableName = ?
"""

#### initializing the database, table,schema details
table_name = 'ModelRunMeta'
database_name = 'test'
schema_name = 'dbo'


#### creating the parameters list
params = (database_name,schema_name,table_name)

### calling the delete function
delete(query,cursor,params)

#### using SQL alchemy

In [22]:
#### connection of python to SQL server using SQLalchemy configuration
def connect_to_sql_server(server_name, database_name):

    # Connect using sql alchemy
    driver_name = 'SQL Server Native Client 11.0'
    params = urllib.parse.quote_plus('''Driver={};
                                     Server={};
                                     Database={};
                                    Trusted_Connection=yes'''
                                    .format(driver_name, server_name,
                                             database_name))

    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

    return engine

In [11]:
table_name = 'ModelRunMeta'
database_name = 'test'
schema_name = 'dbo'
server = 'AAPL-L-627\SQLEXPRESS'

In [24]:
### making the connection
engine = connect_to_sql_server(server, database_name)

In [25]:
def delete(engine):
    with engine.begin() as connect:
        connect.execute("exec dbo.del ?, ?, ?",[database_name,schema_name,table_name])

In [26]:
delete(engine)

#### 2nd method using SQL alchemy

In [15]:
with conn.connect().execution_options(autocommit=True) as con:
    con.execute(txt("EXEC del @DBName = " + database_name + ", @SchemaName =" + schema_name + ", @TableName =" + table_name))